-----

## Setup

In [88]:
use Math::Matrix;
use Math::Splines;
use Math::Fitting;
use Math::DistanceFunctions;

use Data::Reshapers;
use Data::Summarizers;
use Data::Generators;
use Data::Importers;

use JavaScript::D3;
use JavaScript::Google::Charts;

use Hash::Merge;
use LLM::Configurations;

use Mathematica::Serializer;

### Google Charts

In [ ]:
#% javascript
google.charts.load('current', {'packages':['corechart']});
google.charts.load('current', {'packages':['gauge']});
google.charts.load('current', {'packages':['wordtree']});
google.charts.load('current', {'packages':['geochart']});
google.charts.load('current', {'packages':['table']});
google.charts.load('current', {'packages':['line']});
google.charts.setOnLoadCallback(function() {
    console.log('Google Charts library loaded');
});


#### Dark mode

In [ ]:
my $format = 'html';
my $titleTextStyle = { color => 'Ivory', fontSize => 16 };
my $backgroundColor = '#1F1F1F';
my $legendTextStyle = { color => 'Silver' };
my $legend = { position => "none", textStyle => {fontSize => 14, color => 'Silver'} };

my $hAxis = { title => 'x', titleTextStyle => { color => 'Silver' }, textStyle => { color => 'Gray'}, logScale => False, format => 'decimal'};
my $vAxis = { title => 'y', titleTextStyle => { color => 'Silver' }, textStyle => { color => 'Gray'}, logScale => False, format => 'decimal'};

my $annotations = {textStyle => {color => 'Silver', fontSize => 10}};
my $chartArea = {left => 50, right => 50, top => 50, bottom => 50, width => '90%', height => '90%'};

------

## Interpolation

In [ ]:
my @points = [[1,2],[0,1],[2,0],[2,2],[3,3],[5,2]];

In [ ]:
my @knots = 0, 0, 0, 0, 1/3, 2/3, 1, 1, 1, 1;

In [ ]:
my @control-points = [[1.0,2.0],[-1.58322,1.80698],[3.44696,-2.0377],[0.649963,3.545],[4.99045,3.20623],[5.0,2.0]];

In [ ]:
my ($min, $max) = @points».head.&{(.min, .max)}();

In [ ]:
my @fit = (0, 0.01 ... 1).map({ b-spline-curve-value(@control-points, :@knots, argument => $_) }).Array;

In [ ]:
@fit ==> encode-to-wl()

In [ ]:
#%html
js-google-charts('LineChart', @fit, 
    title => "B-spline curve", 
    :$titleTextStyle, :$backgroundColor, :$chartArea, :$hAxis, :$vAxis,
    lineWidth => 4,
    width => 600, 
    height => 400, 
    div-id => 'spline2', :$format,
    :png-button)

In [ ]:
my @plotData = @fit.map( -> $p { $p.clone.push( @points.map( -> $x { euclidean-distance($p, $x) }).min < 0.04 ?? $p.tail !! NaN )});
@plotData = @plotData.map({ <x fit data>.Array Z=> $_.Array })».Hash;

deduce-type(@plotData):tally

In [ ]:
#% html

js-google-charts('ComboChart', 
    @plotData,
    title => 'Interpolated',
    column-names => <x fit data>,
    :$backgroundColor, :$titleTextStyle,
    :$hAxis, :$vAxis,
    seriesType => 'scatter',
    series => {
        0 => {type => 'line', lineWidth => 4},
        1 => {type => 'scatter', pointSize => 3, opacity => 0.1, color => 'Red'},
    },
    legend => merge-hash($legend, %(position => 'bottom')),
    :$chartArea,
    width => 800, 
    height => 400, 
    div-id => 'interp', :$format,
    :png-button)